In [ ]:
import os
import re
import pdfplumber
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# ========== CONFIGURATION ==========
# Set these paths before running the notebook
INPUT_DIR = "."    # Directory containing payslip PDFs
OUTPUT_DIR = "."   # Directory for generated reports and CSV
# ===================================

## 📁 Data Extraction

Extract all relevant data from PDF payslips including:
- Net salary (Netto del mese)
- Gross salary (Totale competenze)
- Deductions (Totale trattenute)
- TFR (Trattamento di Fine Rapporto)
- Ferie (Vacation days: accrued, used, remaining)
- Permessi P.A.R. (Leave hours)

In [ ]:
def parse_italian_number(value):
    """Convert Italian number format (1.234,56) to float"""
    if not value:
        return None
    try:
        # Remove dots (thousands separator) and replace comma with dot
        cleaned = value.replace('.', '').replace(',', '.')
        return float(cleaned)
    except:
        return None

def extract_payslip_data(pdf_path):
    """Extract all relevant data from a payslip PDF"""
    data = {'file': os.path.basename(pdf_path)}
    
    # Extract date from filename
    match = re.search(r'(\d{2})-(\d{4})', pdf_path)
    if match:
        month, year = match.groups()
        data['date'] = datetime(int(year), int(month), 1)
        data['month'] = int(month)
        data['year'] = int(year)
    
    # Categorize payslip type
    is_aggiuntive = 'aggiuntive' in pdf_path.lower()
    data['is_tredicesima'] = is_aggiuntive  # 13th month salary
    data['payslip_type'] = 'tredicesima' if is_aggiuntive else 'regular'
    
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text() or ''
    
    # Net salary (total received)
    match = re.search(r'NETTO.?DEL.?MESE\s*\n?\s*([\d.,]+)\s*€', text, re.IGNORECASE)
    data['netto'] = parse_italian_number(match.group(1)) if match else None
    
    # Retribuzione (actual salary for days worked) - code Z00001
    # Pattern: Z00001Retribuzione followed by rate, days, then amount
    match = re.search(r'Z00001\s*Retribuzione.*?([\d.,]+)\s*$', text, re.IGNORECASE | re.MULTILINE)
    if match:
        data['retribuzione'] = parse_italian_number(match.group(1))
    else:
        # Alternative: look for just the pattern with GG (giorni)
        match = re.search(r'Z00001\s*Retribuzione\s+[\d.,]+\s+[\d.,]+\s*GG\s+([\d.,]+)', text, re.IGNORECASE)
        data['retribuzione'] = parse_italian_number(match.group(1)) if match else None
    
    # Bonus Aziendale (code 500030) - one-time company bonus (GROSS amount)
    match = re.search(r'500030\s*Bonus\s*Aziendale\s+([\d.,]+)', text, re.IGNORECASE)
    data['bonus_aziendale'] = parse_italian_number(match.group(1)) if match else 0.0
    
    # Rimborso Spese (code 000070) - expense reimbursements (not taxed, not part of salary)
    match = re.search(r'000070\s*Rimborso\s*spese\s+([\d.,]+)', text, re.IGNORECASE)
    data['rimborso_spese'] = parse_italian_number(match.group(1)) if match else 0.0
    
    # Mark if this payslip has bonus aziendale
    data['has_bonus_aziendale'] = data['bonus_aziendale'] > 0
    
    # Gross salary (Totale Competenze)
    match = re.search(r'TOTALE.?COMPETENZE\s+([\d.,]+)', text, re.IGNORECASE)
    data['lordo'] = parse_italian_number(match.group(1)) if match else None
    
    # Total deductions (Totale Trattenute)
    match = re.search(r'TOTALE.?TRATTENUTE\s+([\d.,]+)', text, re.IGNORECASE)
    data['trattenute'] = parse_italian_number(match.group(1)) if match else None
    
    # Calculate netto_stipendio: net salary without rimborsi (which aren't taxed)
    # For months WITH bonus: estimate using the tax rate applied to total
    # For months WITHOUT bonus: just subtract rimborsi
    if data['netto'] and data['lordo'] and data['trattenute']:
        # Tax rate for this payslip
        tax_rate = data['trattenute'] / data['lordo'] if data['lordo'] > 0 else 0
        
        if data['bonus_aziendale'] > 0:
            # Estimate what the lordo would be without bonus
            lordo_without_bonus = data['lordo'] - data['bonus_aziendale']
            # Apply same tax rate to estimate net without bonus
            estimated_trattenute = lordo_without_bonus * tax_rate
            data['netto_stipendio'] = lordo_without_bonus - estimated_trattenute - data['rimborso_spese']
        else:
            # No bonus - just subtract rimborsi from net
            data['netto_stipendio'] = data['netto'] - data['rimborso_spese']
    else:
        data['netto_stipendio'] = data['netto'] - data['rimborso_spese'] if data['netto'] else None
    
    # TFR quota for the month
    match = re.search(r'Quota\s+T\.?F\.?R\.?\s+([\d.,]+)', text, re.IGNORECASE)
    data['tfr_quota'] = parse_italian_number(match.group(1)) if match else None
    
    # TFR accumulated fund
    match = re.search(r'T\.?F\.?R\.?\s+F\.?do\s+31/12\s+([\d.,]+)', text, re.IGNORECASE)
    data['tfr_fondo'] = parse_italian_number(match.group(1)) if match else None
    
    # Ferie (vacation) - Maturato, Goduto, Residuo
    match = re.search(r'Ferie\s+([\d.,-]+)\s+([\d.,-]+)\s+([\d.,-]+)', text, re.IGNORECASE)
    if match:
        data['ferie_maturate'] = parse_italian_number(match.group(1))
        data['ferie_godute'] = parse_italian_number(match.group(2))
        data['ferie_residue'] = parse_italian_number(match.group(3))
    
    # Permessi P.A.R. - Maturato, Goduto, Residuo
    match = re.search(r'Perm\.?P\.?A\.?R\s+([\d.,-]+)\s+([\d.,-]+)\s+([\d.,-]+)', text, re.IGNORECASE)
    if match:
        data['permessi_maturati'] = parse_italian_number(match.group(1))
        data['permessi_goduti'] = parse_italian_number(match.group(2))
        data['permessi_residui'] = parse_italian_number(match.group(3))
    
    # IRPEF
    match = re.search(r'IRPEF\s+lorda\s+([\d.,]+)', text, re.IGNORECASE)
    data['irpef'] = parse_italian_number(match.group(1)) if match else None
    
    # Imponibile IRPEF
    match = re.search(r'Imponibile\s+IRPEF\s+([\d.,]+)', text, re.IGNORECASE)
    data['imponibile_irpef'] = parse_italian_number(match.group(1)) if match else None
    
    # Worked hours
    match = re.search(r'Ore\s*ordinarie\s+([\d.,]+)', text, re.IGNORECASE)
    data['ore_ordinarie'] = parse_italian_number(match.group(1)) if match else None
    
    return data

# Load all payslips from INPUT_DIR
print(f"📂 Loading payslips from: {os.path.abspath(INPUT_DIR)}")
pdf_files = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith('.pdf') and f.startswith('Libro_unico')])
print(f"Found {len(pdf_files)} payslip files")

payslips = []
for pdf_file in pdf_files:
    try:
        pdf_path = os.path.join(INPUT_DIR, pdf_file)
        data = extract_payslip_data(pdf_path)
        payslips.append(data)
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")

df = pd.DataFrame(payslips)
df = df.sort_values('date').reset_index(drop=True)
print(f"✅ Successfully processed {len(df)} payslips")

# Dataset summary
print(f"\n📅 Date range: {df['date'].min().strftime('%B %Y')} - {df['date'].max().strftime('%B %Y')}")

# Show breakdown by type
n_regular = len(df[~df['is_tredicesima']])
n_tredicesima = len(df[df['is_tredicesima']])
n_with_bonus = len(df[df['has_bonus_aziendale']])
n_with_rimborso = len(df[df['rimborso_spese'] > 0])
print(f"\n📊 Payslip breakdown:")
print(f"   Regular payslips:              {n_regular}")
print(f"   Tredicesima (13th month):      {n_tredicesima}")
print(f"   Payslips with Bonus Aziendale: {n_with_bonus}")
print(f"   Payslips with Rimborso Spese:  {n_with_rimborso}")

# Show notable payslips (those with bonus or high amounts)
df_notable = df[(df['has_bonus_aziendale']) | (df['is_tredicesima'])].copy()
if len(df_notable) > 0:
    print(f"\n📋 Notable payslips (bonus/tredicesima):")
    for _, row in df_notable.iterrows():
        label = row['date'].strftime('%B %Y')
        extras = []
        if row['is_tredicesima']:
            extras.append("Tredicesima")
        if row['has_bonus_aziendale']:
            extras.append(f"Bonus €{row['bonus_aziendale']:,.2f}")
        print(f"   {label}: €{row['netto']:,.2f} net ({', '.join(extras)})")

## 📋 Overview Table

Complete payslip data ordered by date.

In [ ]:
# Display main columns with new categorization
display_cols = ['date', 'payslip_type', 'retribuzione', 'bonus_aziendale', 'rimborso_spese', 'netto', 'netto_stipendio', 'lordo', 'trattenute']
available_cols = [c for c in display_cols if c in df.columns]
df[available_cols].style.format({
    'retribuzione': '€{:,.2f}',
    'netto': '€{:,.2f}',
    'netto_stipendio': '€{:,.2f}',
    'bonus_aziendale': '€{:,.2f}',
    'rimborso_spese': '€{:,.2f}',
    'lordo': '€{:,.2f}',
    'trattenute': '€{:,.2f}',
    'date': lambda x: x.strftime('%b %Y') if pd.notna(x) else ''
}).background_gradient(subset=['retribuzione'], cmap='Greens')

## 💰 Salary Statistics

In [ ]:
# Separate regular payslips from tredicesima
df_regular = df[~df['is_tredicesima']].copy()
df_tredicesima = df[df['is_tredicesima']].copy()

print("=" * 60)
print("📊 SALARY STATISTICS")
print("=" * 60)

print(f"\n💵 True Net Salary (excluding rimborsi e bonus):")
print(f"   Average:  €{df_regular['netto_stipendio'].mean():,.2f}")
print(f"   Median:   €{df_regular['netto_stipendio'].median():,.2f}")
print(f"   Min:      €{df_regular['netto_stipendio'].min():,.2f}")
print(f"   Max:      €{df_regular['netto_stipendio'].max():,.2f}")

print(f"\n💵 Total Net Received (including everything):")
print(f"   Average:  €{df_regular['netto'].mean():,.2f}")

# Rimborsi summary
total_rimborsi = df['rimborso_spese'].sum()
if total_rimborsi > 0:
    print(f"\n🚗 Total Rimborso Spese: €{total_rimborsi:,.2f}")
    months_with_rimborsi = len(df[df['rimborso_spese'] > 0])
    print(f"   In {months_with_rimborsi} months")

# Bonus aziendale summary  
total_bonus = df['bonus_aziendale'].sum()
if total_bonus > 0:
    print(f"\n🎁 Total Bonus Aziendale: €{total_bonus:,.2f}")

# Tredicesima summary
if len(df_tredicesima) > 0:
    print(f"\n🎄 Tredicesima (13th month salaries):")
    print(f"   Count: {len(df_tredicesima)}")
    print(f"   Average: €{df_tredicesima['netto'].mean():,.2f}")
    print(f"   Total: €{df_tredicesima['netto'].sum():,.2f}")

if 'lordo' in df.columns and df_regular['lordo'].notna().any():
    print(f"\n💼 Gross Salary (Lordo):")
    print(f"   Average:  €{df_regular['lordo'].mean():,.2f}")
    print(f"   Min:      €{df_regular['lordo'].min():,.2f}")
    print(f"   Max:      €{df_regular['lordo'].max():,.2f}")

if 'trattenute' in df.columns and df_regular['trattenute'].notna().any():
    print(f"\n📉 Average Deductions: €{df_regular['trattenute'].mean():,.2f}")
    avg_deduction_pct = (df_regular['trattenute'] / df_regular['lordo'] * 100).mean()
    print(f"   Average Deduction Rate: {avg_deduction_pct:.1f}%")

print(f"\n📅 Period: {df['date'].min().strftime('%B %Y')} - {df['date'].max().strftime('%B %Y')}")
print(f"   Total: {len(df_regular)} regular + {len(df_tredicesima)} tredicesima")

## 📈 Net Salary Trend

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot true net salary (regular months)
ax.plot(df_regular['date'], df_regular['netto_stipendio'], 'o-', color='#2ecc71', 
        linewidth=2, markersize=8, label='Net Salary (stipendio)')

# Highlight months with rimborso spese
df_with_rimborso = df_regular[df_regular['rimborso_spese'] > 0]
if len(df_with_rimborso) > 0:
    ax.scatter(df_with_rimborso['date'], df_with_rimborso['netto'], color='#3498db', 
               s=100, marker='s', alpha=0.6, zorder=4, label='Total with Rimborsi')

# Highlight months with bonus aziendale
df_with_bonus = df_regular[df_regular['has_bonus_aziendale']]
if len(df_with_bonus) > 0:
    ax.scatter(df_with_bonus['date'], df_with_bonus['netto'], color='#9b59b6', 
               s=150, marker='D', zorder=5, label='Total with Bonus Aziendale')

# Plot tredicesima as separate markers
if len(df_tredicesima) > 0:
    ax.scatter(df_tredicesima['date'], df_tredicesima['netto'], color='#e74c3c', 
               s=150, marker='*', zorder=5, label='Tredicesima (13th month)')

# Add average line for true salary
avg_netto = df_regular['netto_stipendio'].mean()
ax.axhline(y=avg_netto, color='#95a5a6', linestyle='--', alpha=0.7, 
           label=f'Avg Salary: €{avg_netto:,.0f}')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Amount (€)', fontsize=12)
ax.set_title('💰 Net Salary Trend Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 📊 Year-over-Year Comparison

In [ ]:
# Group by year - using true net salary
yearly_stats = df_regular.groupby('year').agg({
    'netto_stipendio': ['sum', 'mean', 'count'],
    'netto': 'sum',
    'lordo': 'sum',
    'tfr_quota': 'sum',
    'bonus_aziendale': 'sum',
    'rimborso_spese': 'sum'
}).round(2)

yearly_stats.columns = ['Salary Total', 'Salary Avg', 'Months', 'Net Total (incl extras)', 'Gross Total', 'TFR Total', 'Bonus Aziendale', 'Rimborsi']

# Add tredicesima totals
tredicesima_by_year = df_tredicesima.groupby('year')['netto'].sum()
yearly_stats['Tredicesima'] = tredicesima_by_year
yearly_stats['Grand Total'] = yearly_stats['Salary Total'] + yearly_stats['Tredicesima'].fillna(0) + yearly_stats['Bonus Aziendale'].fillna(0)

print("📅 YEARLY SUMMARY")
print("=" * 70)
print("Note: 'Salary' excludes rimborsi spese and bonus aziendale")
yearly_stats.style.format({
    'Salary Total': '€{:,.2f}',
    'Salary Avg': '€{:,.2f}',
    'Net Total (incl extras)': '€{:,.2f}',
    'Gross Total': '€{:,.2f}',
    'TFR Total': '€{:,.2f}',
    'Bonus Aziendale': '€{:,.2f}',
    'Rimborsi': '€{:,.2f}',
    'Tredicesima': '€{:,.2f}',
    'Grand Total': '€{:,.2f}'
})

In [ ]:
# Year comparison bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

years = yearly_stats.index.tolist()
x = np.arange(len(years))
width = 0.25

# Total earnings by year - stacked bar
ax1 = axes[0]
bars1 = ax1.bar(x - width, yearly_stats['Salary Total'], width, label='Regular Salary', color='#2ecc71')
bars2 = ax1.bar(x, yearly_stats['Tredicesima'].fillna(0), width, label='Tredicesima', color='#e74c3c')
bars3 = ax1.bar(x + width, yearly_stats['Bonus Aziendale'].fillna(0), width, label='Bonus Aziendale', color='#9b59b6')

ax1.set_xlabel('Year')
ax1.set_ylabel('Total (€)')
ax1.set_title('💵 Total Earnings by Year', fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(years)
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x/1000:.0f}k'))

# Average monthly salary by year (true salary only)
ax2 = axes[1]
bars = ax2.bar(years, yearly_stats['Salary Avg'], color='#3498db')
ax2.set_xlabel('Year')
ax2.set_ylabel('Average Monthly Net Salary (€)')
ax2.set_title('📈 Average Monthly Net Salary by Year', fontweight='bold')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.annotate(f'€{height:,.0f}', xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3), textcoords='offset points', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 🏖️ Ferie & Permessi (Vacation & Leave)

In [ ]:
if 'ferie_residue' in df.columns and df['ferie_residue'].notna().any():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Ferie trend
    ax1 = axes[0]
    ax1.plot(df_regular['date'], df_regular['ferie_residue'], 'o-', color='#f39c12', 
             linewidth=2, markersize=6)
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax1.fill_between(df_regular['date'], df_regular['ferie_residue'], 0, 
                     where=(df_regular['ferie_residue'] >= 0), alpha=0.3, color='green')
    ax1.fill_between(df_regular['date'], df_regular['ferie_residue'], 0, 
                     where=(df_regular['ferie_residue'] < 0), alpha=0.3, color='red')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Hours')
    ax1.set_title('🏖️ Vacation Hours Balance (Ferie Residue)', fontweight='bold')
    plt.sca(ax1)
    plt.xticks(rotation=45)
    
    # Permessi trend
    ax2 = axes[1]
    if 'permessi_residui' in df.columns and df_regular['permessi_residui'].notna().any():
        ax2.plot(df_regular['date'], df_regular['permessi_residui'], 'o-', color='#9b59b6', 
                 linewidth=2, markersize=6)
        ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        ax2.fill_between(df_regular['date'], df_regular['permessi_residui'], 0, alpha=0.3, color='#9b59b6')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Hours')
    ax2.set_title('⏰ Leave Hours Balance (Permessi P.A.R. Residui)', fontweight='bold')
    plt.sca(ax2)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Current status
    latest = df_regular.iloc[-1]
    print(f"\n📊 Current Leave Status (as of {latest['date'].strftime('%B %Y')}):")
    print(f"   🏖️ Ferie Residue: {latest.get('ferie_residue', 'N/A')} hours")
    print(f"   ⏰ Permessi Residui: {latest.get('permessi_residui', 'N/A')} hours")
else:
    print("Ferie/Permessi data not available in payslips")

## 💼 TFR (Trattamento di Fine Rapporto)

In [ ]:
if 'tfr_quota' in df.columns and df_regular['tfr_quota'].notna().any():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Monthly TFR contributions
    ax1 = axes[0]
    ax1.bar(df_regular['date'], df_regular['tfr_quota'], color='#1abc9c', alpha=0.8)
    ax1.set_xlabel('Date')
    ax1.set_ylabel('TFR Quota (€)')
    ax1.set_title('📊 Monthly TFR Contributions', fontweight='bold')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
    plt.sca(ax1)
    plt.xticks(rotation=45)
    
    # Cumulative TFR
    ax2 = axes[1]
    cumulative_tfr = df_regular['tfr_quota'].cumsum()
    ax2.fill_between(df_regular['date'], cumulative_tfr, alpha=0.3, color='#1abc9c')
    ax2.plot(df_regular['date'], cumulative_tfr, 'o-', color='#1abc9c', linewidth=2)
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Cumulative TFR (€)')
    ax2.set_title('📈 Cumulative TFR Accrued', fontweight='bold')
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
    plt.sca(ax2)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # TFR Summary
    total_tfr = df_regular['tfr_quota'].sum()
    latest_fondo = df_regular['tfr_fondo'].dropna().iloc[-1] if 'tfr_fondo' in df.columns and df_regular['tfr_fondo'].notna().any() else None
    
    print(f"\n💼 TFR Summary:")
    print(f"   Total TFR accrued in period: €{total_tfr:,.2f}")
    print(f"   Average monthly TFR: €{df_regular['tfr_quota'].mean():,.2f}")
    if latest_fondo:
        print(f"   TFR Fund (F.do 31/12): €{latest_fondo:,.2f}")
else:
    print("TFR data not available in payslips")

## 📊 Monthly Breakdown (Gross vs Net vs Deductions)

In [ ]:
if 'lordo' in df.columns and df_regular['lordo'].notna().any():
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(len(df_regular))
    width = 0.25
    
    bars1 = ax.bar(x - width, df_regular['lordo'], width, label='Gross (Lordo)', color='#3498db', alpha=0.8)
    bars2 = ax.bar(x, df_regular['netto'], width, label='Net (Netto)', color='#2ecc71', alpha=0.8)
    bars3 = ax.bar(x + width, df_regular['trattenute'], width, label='Deductions', color='#e74c3c', alpha=0.8)
    
    ax.set_xlabel('Month')
    ax.set_ylabel('Amount (€)')
    ax.set_title('💵 Monthly Breakdown: Gross vs Net vs Deductions', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([d.strftime('%b\n%Y') for d in df_regular['date']], fontsize=8)
    ax.legend()
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
    
    plt.tight_layout()
    plt.show()
else:
    print("Gross salary data not available")

## 🥧 Average Salary Distribution

In [ ]:
if 'lordo' in df.columns and df_regular['lordo'].notna().any():
    avg_lordo = df_regular['lordo'].mean()
    avg_netto = df_regular['netto'].mean()
    avg_trattenute = df_regular['trattenute'].mean()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Pie chart
    ax1 = axes[0]
    sizes = [avg_netto, avg_trattenute]
    labels = [f'Net\n€{avg_netto:,.0f}', f'Deductions\n€{avg_trattenute:,.0f}']
    colors = ['#2ecc71', '#e74c3c']
    explode = (0.05, 0)
    
    ax1.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.set_title(f'Average Salary Distribution\n(Gross: €{avg_lordo:,.0f})', fontweight='bold')
    
    # Breakdown of typical deductions estimate
    ax2 = axes[1]
    if 'irpef' in df.columns and df_regular['irpef'].notna().any():
        avg_irpef = df_regular['irpef'].mean()
        other_deductions = avg_trattenute - avg_irpef
        
        deduction_labels = ['IRPEF', 'Other (INPS, etc.)']
        deduction_sizes = [avg_irpef, max(0, other_deductions)]
        deduction_colors = ['#e74c3c', '#f39c12']
        
        ax2.pie(deduction_sizes, labels=[f'{l}\n€{s:,.0f}' for l, s in zip(deduction_labels, deduction_sizes)], 
                colors=deduction_colors, autopct='%1.1f%%', shadow=True, startangle=90)
        ax2.set_title('Average Deductions Breakdown', fontweight='bold')
    else:
        ax2.text(0.5, 0.5, 'IRPEF breakdown\nnot available', ha='center', va='center', fontsize=12)
        ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

## 📊 Summary Statistics

In [ ]:
print("=" * 70)
print("📊 COMPLETE SUMMARY")
print("=" * 70)

# Calculate totals using correct categories
total_salary = df_regular['netto_stipendio'].sum()  # True salary only
total_tredicesima = df_tredicesima['netto'].sum() if len(df_tredicesima) > 0 else 0
total_bonus_aziendale = df['bonus_aziendale'].sum()
total_rimborsi = df['rimborso_spese'].sum()
total_net_received = df['netto'].sum()  # Everything received

print(f"\n💵 TOTAL EARNINGS:")
print(f"   Net Salary (stipendio):    €{total_salary:,.2f}")
print(f"   Tredicesima (13th month):  €{total_tredicesima:,.2f}")
print(f"   Bonus Aziendale:           €{total_bonus_aziendale:,.2f}")
print(f"   ----------------------------------------")
print(f"   Total Compensation:        €{total_salary + total_tredicesima + total_bonus_aziendale:,.2f}")
print(f"\n🚗 Rimborso Spese (reimbursements, not income): €{total_rimborsi:,.2f}")
print(f"\n💳 TOTAL RECEIVED ON ACCOUNT: €{total_net_received:,.2f}")

if 'lordo' in df.columns:
    total_lordo = df['lordo'].sum()
    total_trattenute = df['trattenute'].sum()
    print(f"\n   Total Gross:          €{total_lordo:,.2f}")
    print(f"   Total Deductions:     €{total_trattenute:,.2f}")

if 'tfr_quota' in df.columns:
    total_tfr = df['tfr_quota'].sum()
    print(f"\n💼 TFR Accrued:          €{total_tfr:,.2f}")

print(f"\n📅 PERIOD:")
print(f"   From: {df['date'].min().strftime('%B %Y')}")
print(f"   To:   {df['date'].max().strftime('%B %Y')}")
print(f"   Duration: {len(df_regular)} regular months + {len(df_tredicesima)} tredicesima")

# Current balances
latest = df_regular.iloc[-1]
print(f"\n🏖️ CURRENT BALANCES (as of {latest['date'].strftime('%B %Y')}):")
if 'ferie_residue' in latest and pd.notna(latest['ferie_residue']):
    print(f"   Vacation Hours:       {latest['ferie_residue']:.1f} hours")
if 'permessi_residui' in latest and pd.notna(latest['permessi_residui']):
    print(f"   Leave Hours:          {latest['permessi_residui']:.1f} hours")

## 💾 Export Data & PDF Report

In [ ]:
# Export to CSV for further analysis
os.makedirs(OUTPUT_DIR, exist_ok=True)
export_df = df.copy()
export_df['date_str'] = export_df['date'].dt.strftime('%Y-%m')
csv_path = os.path.join(OUTPUT_DIR, 'payslip_data.csv')
export_df.to_csv(csv_path, index=False)
print(f"✅ Data exported to {csv_path}")

# Display final dataframe summary
df.describe()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from datetime import date

def export_to_pdf(output_dir, filename='payslip_report.pdf'):
    """Export all analysis charts and summary to a PDF report"""
    
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    
    with PdfPages(filepath) as pdf:
        # Page 1: Title and Summary
        fig, ax = plt.subplots(figsize=(11, 8.5))
        ax.axis('off')
        
        title_text = f"Payslip Analysis Report\n{df['date'].min().strftime('%B %Y')} - {df['date'].max().strftime('%B %Y')}"
        ax.text(0.5, 0.85, title_text, ha='center', va='top', fontsize=24, fontweight='bold')
        
        # Summary statistics
        total_salary = df_regular['netto_stipendio'].sum()
        total_tredicesima = df_tredicesima['netto'].sum() if len(df_tredicesima) > 0 else 0
        total_bonus = df['bonus_aziendale'].sum()
        total_rimborsi = df['rimborso_spese'].sum()
        total_received = df['netto'].sum()
        
        summary_text = f"""
COMPLETE SUMMARY
{'='*50}

TOTAL EARNINGS:
   Net Salary (stipendio):     EUR {total_salary:>12,.2f}
   Tredicesima (13th month):   EUR {total_tredicesima:>12,.2f}
   Bonus Aziendale:            EUR {total_bonus:>12,.2f}
   {'─'*40}
   Total Compensation:         EUR {total_salary + total_tredicesima + total_bonus:>12,.2f}

Rimborso Spese:                EUR {total_rimborsi:>12,.2f}

TOTAL RECEIVED:                EUR {total_received:>12,.2f}

STATISTICS (Regular months):
   Average Net Salary:         EUR {df_regular['netto_stipendio'].mean():>12,.2f}
   Median Net Salary:          EUR {df_regular['netto_stipendio'].median():>12,.2f}
   
TFR Accrued:                   EUR {df['tfr_quota'].sum():>12,.2f}

Period: {len(df_regular)} regular months + {len(df_tredicesima)} tredicesima

Current Vacation Hours:        {df_regular.iloc[-1].get('ferie_residue', 0):.1f} hours
Current Leave Hours:           {df_regular.iloc[-1].get('permessi_residui', 0):.1f} hours

Report generated: {date.today().strftime('%d %B %Y')}
"""
        ax.text(0.1, 0.7, summary_text, ha='left', va='top', fontsize=11, 
                fontfamily='monospace', transform=ax.transAxes)
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
        
        # Page 2: Net Salary Trend
        fig, ax = plt.subplots(figsize=(11, 6))
        ax.plot(df_regular['date'], df_regular['netto_stipendio'], 'o-', color='#2ecc71', 
                linewidth=2, markersize=8, label='Net Salary (stipendio)')
        
        df_with_rimborso = df_regular[df_regular['rimborso_spese'] > 0]
        if len(df_with_rimborso) > 0:
            ax.scatter(df_with_rimborso['date'], df_with_rimborso['netto'], color='#3498db', 
                       s=100, marker='s', alpha=0.6, zorder=4, label='Total with Rimborsi')
        
        df_with_bonus = df_regular[df_regular['has_bonus_aziendale']]
        if len(df_with_bonus) > 0:
            ax.scatter(df_with_bonus['date'], df_with_bonus['netto'], color='#9b59b6', 
                       s=150, marker='D', zorder=5, label='Total with Bonus Aziendale')
        
        if len(df_tredicesima) > 0:
            ax.scatter(df_tredicesima['date'], df_tredicesima['netto'], color='#e74c3c', 
                       s=150, marker='*', zorder=5, label='Tredicesima (13th month)')
        
        avg_netto = df_regular['netto_stipendio'].mean()
        ax.axhline(y=avg_netto, color='#95a5a6', linestyle='--', alpha=0.7, 
                   label=f'Avg Salary: €{avg_netto:,.0f}')
        
        ax.set_xlabel('Date', fontsize=12)
        ax.set_ylabel('Amount (€)', fontsize=12)
        ax.set_title('Net Salary Trend Over Time', fontsize=14, fontweight='bold')
        ax.legend(loc='upper left')
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
        plt.xticks(rotation=45)
        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
        
        # Page 3: Year-over-Year Comparison
        fig, axes = plt.subplots(1, 2, figsize=(11, 5))
        years = yearly_stats.index.tolist()
        x = np.arange(len(years))
        width = 0.25
        
        ax1 = axes[0]
        ax1.bar(x - width, yearly_stats['Salary Total'], width, label='Regular Salary', color='#2ecc71')
        ax1.bar(x, yearly_stats['Tredicesima'].fillna(0), width, label='Tredicesima', color='#e74c3c')
        ax1.bar(x + width, yearly_stats['Bonus Aziendale'].fillna(0), width, label='Bonus Aziendale', color='#9b59b6')
        ax1.set_xlabel('Year')
        ax1.set_ylabel('Total (€)')
        ax1.set_title('Total Earnings by Year', fontweight='bold')
        ax1.set_xticks(x)
        ax1.set_xticklabels(years)
        ax1.legend()
        ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x/1000:.0f}k'))
        
        ax2 = axes[1]
        bars = ax2.bar(years, yearly_stats['Salary Avg'], color='#3498db')
        ax2.set_xlabel('Year')
        ax2.set_ylabel('Average Monthly Net Salary (€)')
        ax2.set_title('Average Monthly Net Salary by Year', fontweight='bold')
        ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
        for bar in bars:
            height = bar.get_height()
            ax2.annotate(f'€{height:,.0f}', xy=(bar.get_x() + bar.get_width()/2, height),
                         xytext=(0, 3), textcoords='offset points', ha='center', va='bottom')
        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
        
        # Page 4: Ferie & Permessi
        if 'ferie_residue' in df.columns and df['ferie_residue'].notna().any():
            fig, axes = plt.subplots(1, 2, figsize=(11, 5))
            
            ax1 = axes[0]
            ax1.plot(df_regular['date'], df_regular['ferie_residue'], 'o-', color='#f39c12', linewidth=2, markersize=6)
            ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
            ax1.fill_between(df_regular['date'], df_regular['ferie_residue'], 0, 
                             where=(df_regular['ferie_residue'] >= 0), alpha=0.3, color='green')
            ax1.fill_between(df_regular['date'], df_regular['ferie_residue'], 0, 
                             where=(df_regular['ferie_residue'] < 0), alpha=0.3, color='red')
            ax1.set_xlabel('Date')
            ax1.set_ylabel('Hours')
            ax1.set_title('Vacation Hours Balance (Ferie Residue)', fontweight='bold')
            plt.sca(ax1)
            plt.xticks(rotation=45)
            
            ax2 = axes[1]
            if 'permessi_residui' in df.columns and df_regular['permessi_residui'].notna().any():
                ax2.plot(df_regular['date'], df_regular['permessi_residui'], 'o-', color='#9b59b6', linewidth=2, markersize=6)
                ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
                ax2.fill_between(df_regular['date'], df_regular['permessi_residui'], 0, alpha=0.3, color='#9b59b6')
            ax2.set_xlabel('Date')
            ax2.set_ylabel('Hours')
            ax2.set_title('Leave Hours Balance (Permessi P.A.R. Residui)', fontweight='bold')
            plt.sca(ax2)
            plt.xticks(rotation=45)
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close()
        
        # Page 5: TFR
        if 'tfr_quota' in df.columns and df_regular['tfr_quota'].notna().any():
            fig, axes = plt.subplots(1, 2, figsize=(11, 5))
            
            ax1 = axes[0]
            ax1.bar(df_regular['date'], df_regular['tfr_quota'], color='#1abc9c', alpha=0.8)
            ax1.set_xlabel('Date')
            ax1.set_ylabel('TFR Quota (€)')
            ax1.set_title('Monthly TFR Contributions', fontweight='bold')
            ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
            plt.sca(ax1)
            plt.xticks(rotation=45)
            
            ax2 = axes[1]
            cumulative_tfr = df_regular['tfr_quota'].cumsum()
            ax2.fill_between(df_regular['date'], cumulative_tfr, alpha=0.3, color='#1abc9c')
            ax2.plot(df_regular['date'], cumulative_tfr, 'o-', color='#1abc9c', linewidth=2)
            ax2.set_xlabel('Date')
            ax2.set_ylabel('Cumulative TFR (€)')
            ax2.set_title('Cumulative TFR Accrued', fontweight='bold')
            ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
            plt.sca(ax2)
            plt.xticks(rotation=45)
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close()
        
        # Page 6: Gross vs Net vs Deductions
        if 'lordo' in df.columns and df_regular['lordo'].notna().any():
            fig, ax = plt.subplots(figsize=(11, 6))
            x = np.arange(len(df_regular))
            width = 0.25
            ax.bar(x - width, df_regular['lordo'], width, label='Gross (Lordo)', color='#3498db', alpha=0.8)
            ax.bar(x, df_regular['netto'], width, label='Net (Netto)', color='#2ecc71', alpha=0.8)
            ax.bar(x + width, df_regular['trattenute'], width, label='Deductions', color='#e74c3c', alpha=0.8)
            ax.set_xlabel('Month')
            ax.set_ylabel('Amount (€)')
            ax.set_title('Monthly Breakdown: Gross vs Net vs Deductions', fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels([d.strftime('%b\n%Y') for d in df_regular['date']], fontsize=7)
            ax.legend()
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'€{x:,.0f}'))
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close()
    
    print(f"✅ PDF report exported to {filepath}")

# Generate the PDF report
export_to_pdf(OUTPUT_DIR, 'payslip_report.pdf')